# 4.4 deepspeed分布式训练简介

## 大模型并行训练简介

大模型的并行训练旨在克服单个 GPU 显存的限制和加速训练过程，通常适用于参数规模较大的模型（如 GPT-3、T5 等）。并行训练主要包括以下几种方法，每种方法适用于不同的场景和模型特性。

---

### **1. 数据并行（Data Parallelism）**

#### **原理**
- 将数据切分成多个小批次，每个 GPU 处理其中一部分。
- 模型副本被复制到每个 GPU。
- 每个 GPU 独立计算梯度，最终通过梯度同步（如 AllReduce 操作）更新参数。

#### **特点**
- **优点**：
  - 实现简单，是最常用的并行方法。
  - 对模型大小没有限制。
- **缺点**：
  - 模型副本需要完整加载到每个 GPU，占用显存。
  - 在超大规模模型中，显存压力较大。

#### **适用场景**
- 参数规模适中，显存可以容纳整个模型的场景。

---

### **2. 模型并行（Model Parallelism）**

#### **原理**
- 将模型切分成不同的部分，将不同部分分配到不同的 GPU。
- 前向传播和后向传播时，数据在模型的不同部分之间传递。

#### **特点**
- **优点**：
  - 不需要复制整个模型，可以支持超大规模模型。
- **缺点**：
  - GPU 之间通信频繁，可能成为性能瓶颈。
  - 实现复杂，切分模型需要精心设计。
  
#### **适用场景**
- 单个 GPU 无法容纳完整模型参数的场景。

#### **具体实现**
- 将 Transformer 的不同层分配到不同的 GPU。
- 常用工具：DeepSpeed 的 Pipeline Parallelism、NVIDIA Megatron-LM。

---

### **3. 张量并行（Tensor Parallelism）**

#### **原理**
- 将模型内部的张量（如权重矩阵）切分为多个子张量，并分配到不同 GPU。
- GPU 之间协作完成矩阵计算。

#### **特点**
- **优点**：
  - 减少了每个 GPU 的显存占用，同时保持模型整体完整性。
- **缺点**：
  - 实现较复杂，需要优化通信操作。
  - 通信开销较高，适合较大批量的训练。

#### **适用场景**
- 参数非常大的模型（如 GPT-3）。
- 需要极致优化显存的场景。

#### **具体实现**
- NVIDIA 的 Megatron-LM 和 Hugging Face Transformers 提供了张量并行的支持。

---

### **4. 管道并行（Pipeline Parallelism）**

#### **原理**
- 将模型分为不同的部分（通常是按层划分），每部分分配到不同的 GPU。
- 数据按照流水线的方式流经每个 GPU。

#### **特点**
- **优点**：
  - 减少每个 GPU 的显存压力。
  - 通过流水线增加计算效率。
- **缺点**：
  - 引入流水线延迟。
  - 实现复杂，需管理数据依赖和同步。

#### **适用场景**
- 模型非常深，层数较多的场景。

#### **具体实现**
- DeepSpeed 的 Pipeline Parallelism。

---

### **5. 混合并行（Hybrid Parallelism）**

#### **原理**
- 将数据并行、模型并行、张量并行和管道并行组合使用，充分利用多 GPU 资源。
- 不同的并行方法在不同维度协同工作。

#### **特点**
- **优点**：
  - 灵活且适应性强，适合超大规模模型。
- **缺点**：
  - 配置复杂，依赖于框架和训练任务。

#### **适用场景**
- 超大规模模型（如 GPT-3 或参数量 >1T）。
- 多机多卡的大型训练环境。

#### **具体实现**
- NVIDIA Megatron-LM 和 DeepSpeed 的混合并行支持。

---

### **6. ZeRO 优化并行（Zero Redundancy Optimizer）**

#### **原理**
- 通过分片存储模型参数、优化器状态和梯度，显著减少每个 GPU 的显存占用。

#### **特点**
- **优点**：
  - 极大降低显存需求。
  - 支持超大规模模型。
- **缺点**：
  - 对 GPU 间通信要求较高。
  - 比数据并行复杂。

#### **适用场景**
- 超大模型的高效训练。

#### **具体实现**
- DeepSpeed 提供的 ZeRO Stage 1/2/3。

---

### **方法对比**

| 并行方法      | 主要优点                      | 主要缺点                      | 适用场景                   |
|---------------|-------------------------------|-------------------------------|---------------------------|
| 数据并行      | 简单高效，易实现              | 模型副本占用大量显存          | 模型规模适中，显存足够     |
| 模型并行      | 支持大模型                    | 通信开销大，切分复杂          | 超大模型，显存有限         |
| 张量并行      | 高效利用显存                  | 实现复杂，通信频繁            | 参数规模极大的模型         |
| 管道并行      | 显存需求降低，适合深模型      | 流水线延迟，数据同步复杂      | 层数多的大型模型           |
| 混合并行      | 灵活适配超大规模模型          | 配置复杂，依赖框架            | 超大规模模型（如 GPT-3）   |
| ZeRO 并行     | 极大节省显存，占用少          | 通信成本高                    | 超大规模模型显存优化       |

---

### **总结**
- **中等规模模型**：优先使用 **数据并行**。
- **单卡显存不足**：采用 **模型并行** 或 **张量并行**。
- **超大规模模型**：使用 **混合并行** 或 DeepSpeed 的 **ZeRO 优化**。

对于现代超大规模模型，通常采用混合并行方法，比如 NVIDIA 的 Megatron-LM 和微软的 DeepSpeed，它们综合了多种并行策略，能够有效利用计算资源并加速训练。如果您有具体的硬件环境或模型需求，可以进一步探讨适合的并行方案！

## deepspeed简介

DeepSpeed 是微软开发的开源深度学习优化库，专为大规模模型训练和推理设计，能够显著提升训练速度、降低显存占用，并支持分布式计算。以下是 DeepSpeed 的关键特点和功能：

---

### **1. 核心特点**

#### **（1）高效分布式训练**
DeepSpeed 提供先进的分布式技术（如 ZeRO 优化器），支持数百亿甚至上万亿参数的模型训练，同时降低单设备显存需求。

#### **（2）显存优化**
通过显存分片（ZeRO）、梯度累积和混合精度训练，DeepSpeed 能够在有限显存的情况下训练大模型。

#### **（3）性能提升**
DeepSpeed 优化了通信和计算过程，可提升多 GPU 分布式训练效率。

#### **（4）灵活性**
与 PyTorch 无缝集成，并兼容 Hugging Face `transformers` 和其他主流深度学习库。

#### **（5）推理优化**
支持高效推理（如量化和张量并行），适合大模型的生产部署。

---

### **2. 核心技术**

#### **（1）ZeRO 优化器**
ZeRO（Zero Redundancy Optimizer）是 DeepSpeed 的核心技术之一，分为 3 个阶段：
- **Stage 1**：分片优化器状态（如动量、方差）。
- **Stage 2**：分片优化器状态和梯度。
- **Stage 3**：分片优化器状态、梯度和模型参数，实现全分片优化。

每个阶段都进一步减少显存需求，Stage 3 可支持超大规模模型（如 GPT-3）。

<img src='img/deepspeed.png' width='600px' />

#### **（2）混合精度训练**
通过 FP16 或 BF16（半精度浮点数）计算，显著减少显存占用并提升计算效率。

#### **（3）数据并行与模型并行**
- 数据并行：将数据划分到多个设备，每个设备计算部分梯度。
- 模型并行：将模型的不同部分分配到多个设备。
- 张量并行：将张量运算分解到多个 GPU 上。

#### **（4）梯度累积**
支持更大的有效批量大小，适合显存受限的设备。

#### **（5）推理优化**
- 推理阶段的显存优化和加速技术。
- 量化推理，减少模型大小和运行时开销。

---

### **3. 适用场景**

#### **（1）大规模模型训练**
适合训练数十亿或上万亿参数的模型，如 GPT-3、BERT、T5 等。

#### **（2）分布式训练**
支持单机多卡、多机多卡分布式训练，能高效利用多 GPU 环境。

#### **（3）显存受限的模型微调**
通过显存优化技术，能在较小显存设备（如 16GB GPU）上微调大模型。

#### **（4）高效推理**
适用于大语言模型的生产部署，支持推理加速和量化。

---

### **4. 优势与局限性**

#### **优势**
1. 显存需求显著降低，适合超大规模模型训练。
2. 支持多种分布式模式，扩展性强。
3. 与 PyTorch 和 Hugging Face 无缝集成。
4. 推理优化技术降低部署成本。

#### **局限性**
1. 配置和调优可能较为复杂。
2. 对小规模模型或数据集的性能提升有限。

---

### **5. 安装与基本用法**

#### **安装**
```bash
pip install deepspeed
```

#### **基本用法**
DeepSpeed 通过配置文件启用特性，例如 ZeRO 优化器：
```python
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer
import deepspeed

# 配置 DeepSpeed
deepspeed_config = {
    "train_batch_size": 64,
    "gradient_accumulation_steps": 8,
    "fp16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 2,
        "overlap_comm": True
    }
}

# 保存配置文件
import json
with open("deepspeed_config.json", "w") as f:
    json.dump(deepspeed_config, f)

# 集成到 Hugging Face Trainer
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    fp16=True,
    deepspeed="./deepspeed_config.json"  # DeepSpeed 配置文件
)

trainer = Trainer(
    model=GPT2LMHeadModel.from_pretrained("gpt2"),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

trainer.train()
```

---

### **6. 总结**

DeepSpeed 是大模型训练的强力工具，特别是在多 GPU 环境下，其显存优化和分布式训练技术能显著提升训练效率。适用于以下场景：
- 超大规模模型的训练和微调。
- 多机多卡环境的分布式训练。
- 高效推理部署。

如果需要进一步优化模型训练或部署性能，DeepSpeed 是值得尝试的工具！

## deepspeed具体设置


是的，DeepSpeed 支持多种并行策略，包括 **数据并行**、**模型并行** 和 **张量并行**，并且可以通过其配置文件灵活地设置这些并行模式。

---

### **1. 数据并行**

#### **原理**
在数据并行中，DeepSpeed 将数据批次划分到多个 GPU，每个 GPU 上都有模型的完整副本，计算独立的梯度。最终通过 `AllReduce` 操作同步梯度并更新模型参数。

#### **如何设置**
DeepSpeed 默认支持数据并行，启用 `zero_optimization` 后会自动结合 ZeRO 优化器进行分片数据并行：
```json
{
    "train_batch_size": 64,
    "gradient_accumulation_steps": 8,
    "fp16": {
        "enabled": true
    },
    "zero_optimization": {
        "stage": 1
    }
}
```

---

### **2. 模型并行**

#### **原理**
模型并行将模型的不同部分（如 Transformer 层或权重张量）分布到多个 GPU。DeepSpeed 本身不直接实现模型并行，但可以与模型并行框架（如 NVIDIA Megatron-LM）集成。

#### **如何设置**
如果使用模型并行（如层级分割）：
1. 使用 DeepSpeed 的 Pipeline Parallelism：
   ```json
   {
       "train_batch_size": 64,
       "pipeline_parallel_size": 2,  # 设置流水线并行 GPU 数量
       "fp16": {
           "enabled": true
       },
       "zero_optimization": {
           "stage": 1
       }
   }
   ```

2. 与 NVIDIA Megatron-LM 集成：
   在代码中使用 Megatron-LM 的模型并行支持，然后结合 DeepSpeed：
   ```python
   from megatron import get_model_parallel_world_size
   import deepspeed

   model = MyModel(...)
   model = deepspeed.initialize(
       model=model,
       model_parallel_size=get_model_parallel_world_size(),
       config="./deepspeed_config.json"
   )
   ```

---

### **3. 张量并行**

#### **原理**
张量并行将模型参数张量（如权重矩阵）分片到多个 GPU，并通过通信协作完成计算。DeepSpeed 提供了张量并行的支持（在 ZeRO Stage 3 中），或者通过集成 Megatron-LM 实现。

#### **如何设置**
1. **使用 ZeRO Stage 3**：
   ZeRO Stage 3 会分片模型参数和优化器状态，类似于张量并行的效果：
   ```json
   {
       "train_batch_size": 64,
       "gradient_accumulation_steps": 8,
       "fp16": {
           "enabled": true
       },
       "zero_optimization": {
           "stage": 3,
           "offload_optimizer": {
               "device": "cpu",
               "pin_memory": true
           },
           "offload_param": {
               "device": "cpu",
               "pin_memory": true
           }
       }
   }
   ```

2. **集成 Megatron-LM**：
   如果需要更复杂的张量并行方案（如矩阵切分），可以通过 Megatron-LM 实现，然后与 DeepSpeed 集成。

---

### **4. 混合并行**

#### **原理**
混合并行结合了数据并行、模型并行和张量并行。DeepSpeed 提供了对这些模式的集成支持，允许您灵活配置。

#### **如何设置**
结合数据并行和流水线并行：
```json
{
    "train_batch_size": 64,
    "gradient_accumulation_steps": 8,
    "fp16": {
        "enabled": true
    },
    "pipeline_parallel_size": 2,  # 流水线并行
    "zero_optimization": {
        "stage": 2
    }
}
```

与张量并行结合：
1. 在代码中配置张量并行：
   ```python
   from megatron import get_tensor_parallel_world_size
   model = MyModel(...)
   model = deepspeed.initialize(
       model=model,
       tensor_parallel_size=get_tensor_parallel_world_size(),
       config="./deepspeed_config.json"
   )
   ```

2. DeepSpeed 配置文件中启用 ZeRO Stage 3。

---

### **5. 选择并行策略**

| 并行模式      | **支持方式**                               | **适用场景**                            |
|---------------|------------------------------------------|-----------------------------------------|
| 数据并行      | 默认支持，结合 ZeRO 优化器                | 模型参数较小，显存压力不大的场景         |
| 模型并行      | 使用 Pipeline Parallelism 或集成 Megatron-LM | 模型参数非常大，单 GPU 无法容纳的场景    |
| 张量并行      | ZeRO Stage 3 或集成 Megatron-LM          | 参数矩阵非常大，需要分片计算的场景       |
| 混合并行      | 结合数据并行、模型并行和张量并行          | 超大规模模型（如 GPT-3）训练             |

---

### **6. 示例代码**

以下是集成 ZeRO 和 Pipeline Parallelism 的完整示例：
```python
import deepspeed
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer
from datasets import load_dataset

# 加载数据
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# 加载模型
model = GPT2LMHeadModel.from_pretrained("gpt2")

# 配置 DeepSpeed
deepspeed_config = {
    "train_batch_size": 64,
    "gradient_accumulation_steps": 8,
    "pipeline_parallel_size": 2,  # 流水线并行
    "fp16": {
        "enabled": True
    },
    "zero_optimization": {
        "stage": 2
    }
}

# 保存配置文件
import json
with open("deepspeed_config.json", "w") as f:
    json.dump(deepspeed_config, f)

# 训练参数
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    deepspeed="./deepspeed_config.json",  # 指定 DeepSpeed 配置文件
)

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# 开始训练
trainer.train()
```

---

### **总结**

- **数据并行**：默认支持，结合 ZeRO 进行优化。
- **模型并行**：使用 Pipeline Parallelism 或与 Megatron-LM 集成。
- **张量并行**：通过 ZeRO Stage 3 或 Megatron-LM 实现。
- **混合并行**：灵活结合多种并行方法，用于超大规模模型。

DeepSpeed 的配置高度灵活，可以根据模型大小、显存限制和硬件条件选择适合的并行策略。

## 使用gpt2+deepspeed训练

## gpt2的训练

```
#一般方式训练gpt2
python pretain_gpt2.py


#deepspeed训练gpt2, 只多一行代码
torchrun --nproc_per_node=6 deepspeed_pretrain_gpt2.py

```